In [5]:
# conda create -n spiders-env python=3.11 -y
# conda activate spiders-env

!conda install pip -y
!pip install -r ../requirements.txt

Channels:
 - defaults
Platform: osx-64
Solving environment: done

# All requested packages already installed.

  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata (33 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached cryptography-42.0.8-cp39-abi3-macosx_10_12_universal2.whl.metadata (5.3 kB)
  Using cached numpy-2.0.0-cp312-cp312-macosx_10_9_x86_64.whl.metadata (60 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached selenium-4.22.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached trio-0.26.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using ca

In [21]:
import os
import json
import requests

import pandas as pd
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from scrapy import Selector

from tqdm.notebook import tqdm
from pprint import pprint as print

In [131]:
# Setting program-level variables
driver = webdriver.Chrome()
url = "https://roadtonationals.com/results/standings/"
years = [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015] # These are the years that we are interested in evaluating

print('honk')

'honk'


In [132]:
# PART 1: Go to the url, wait until everything on the page loads

def get_url_and_wait_for_elements_to_load(url, css_selector):
    try:
        driver.get(url)
        print("*****************************")
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)) # (Source: https://selenium-python.readthedocs.io/waits.html )
        )
        print(element)
    except:
        print("oh no it didn't work")
        pass # ?? Trying to make it so that the program doesn't crash if the element isn't found
    

get_url_and_wait_for_elements_to_load(url, 'div.rt-table > div.rt-tbody')
response = Selector(text=driver.page_source)

print(response)

'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.F2F0A140D0AB90353F48C2CDCCFC3BC5.e.14")>
<Selector query=None data='<html class="whatinput-types-initial"...'>


In [133]:
# PART 2: Set up the dataframes we exported in the last session as variables

teams_df = pd.read_csv('../Data/Raw/teams.csv')
meets_df = pd.read_csv('../Data/Raw/meets.csv')

#print(teams_df)
for meet in meets_df['link'][0:20]:
    print(meet)

'https://roadtonationals.com/results/schedule/meet/29998'
'https://roadtonationals.com/results/schedule/meet/29057'
'https://roadtonationals.com/results/schedule/meet/29134'
'https://roadtonationals.com/results/schedule/meet/29224'
'https://roadtonationals.com/results/schedule/meet/29291'
'https://roadtonationals.com/results/schedule/meet/29423'
'https://roadtonationals.com/results/schedule/meet/29515'
'https://roadtonationals.com/results/schedule/meet/29616'
'https://roadtonationals.com/results/schedule/meet/29671'
'https://roadtonationals.com/results/schedule/meet/29810'
'https://roadtonationals.com/results/schedule/meet/29840'
'https://roadtonationals.com/results/schedule/meet/29871'
'https://roadtonationals.com/results/schedule/meet/29949'
'https://roadtonationals.com/results/schedule/meet/30143'
'https://roadtonationals.com/results/schedule/meet/30171'
'https://roadtonationals.com/results/schedule/meet/30206'
'https://roadtonationals.com/results/schedule/meet/30226'
'https://roadt

In [134]:
css_selector = 'div.rt-tbody'
get_url_and_wait_for_elements_to_load(meet_link, css_selector)
response = Selector(text=driver.page_source)  
meet_results_table = response.css(css_selector)
meet_results_table_rows = meet_results_table.css('div.rt-tr-group')
teams_button_clicker = driver.find_element(By.CSS_SELECTOR, '#teambtn')
row_count = 0
meet_host = ''

if response.css('p:nth-child(4)').get():
    meet_host = response.css('p:nth-child(4)::text').get()
else:
    meet_host = 'NaN'

meet_hosts.append(meet_host)


'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.419B255BDF322B9765ED112CE6060F9E.e.15")>


In [135]:
get_url_and_wait_for_elements_to_load(meet_link, css_selector)
response = Selector(text=driver.page_source)

meet_results_table = response.css(css_selector)
meet_results_table.css('div.rt-tr-group::text').getall()

meet_results_table_rows.css('div:nth-child(4)::text').getall()
len(meet_results_table_rows)

'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.6F69F191DAF84E72334C4147AB188957.e.17")>


2

In [136]:
def get_team_score_info(url):

    #Load the page and set the selectors
    get_url_and_wait_for_elements_to_load(url, css_selector)
    response = Selector(text=driver.page_source)  
    meet_results_table_rows = meet_results_table.css('div.rt-tr-group')

    # Determind the meet id
    meet_id = url.split('/')[-1]
    meet_ids.append(meet_id)
    
    # Find out if there is a host for the meet, and if so add to the meet_hosts list
    meet_host = ''
    
    if response.css('p:nth-child(4)').get():
        meet_host = response.css('p:nth-child(4)::text').get()
    else:
        meet_host = 'NaN'

    meet_hosts.append(meet_host)
    
    # Find out how many teams are competing in the meet
    team_count = len(meet_results_table_rows)

    # Add the meet id to the list of team_meet_ids list for each team
    for team_meet_id in range(0, team_count):
        team_meet_id = meet_id
        team_meet_ids.append(team_meet_id)
    
    # Get the hrefs for each team
    team_hrefs = meet_results_table_rows.css('div > div > a::attr(href)').getall()
    # Splitting out the team_id and adding them to the team_ids list
    for team_href in team_hrefs:
        team_id = team_href.split('/')[-1]
        team_ids.append(team_id)

    # Get the scores for each event and the total meet score (this generates a list of lists)
    current_meet_team_vt_scores = meet_results_table_rows.css('div:nth-child(4)::text').getall()
    current_meet_team_ub_scores = meet_results_table_rows.css('div:nth-child(5)::text').getall()
    current_meet_team_bb_scores = meet_results_table_rows.css('div:nth-child(6)::text').getall()
    current_meet_team_fx_scores = meet_results_table_rows.css('div:nth-child(7)::text').getall()
    current_meet_team_meet_scores = meet_results_table_rows.css('div:nth-child(8) > strong::text').getall()

    # Iterating over the lists generated above and adding them to the appropriate (variable) list
    for score in current_meet_team_vt_scores:
        team_vt_scores.append(score)
    
    for score in current_meet_team_ub_scores:
        team_ub_scores.append(score)
    
    for score in current_meet_team_bb_scores:
        team_bb_scores.append(score)
    
    for score in current_meet_team_fx_scores:
        team_fx_scores.append(score)
    
    for score in current_meet_team_meet_scores:
        team_meet_scores.append(score)
    
    
    return team_ids, team_meet_ids, team_vt_scores, team_ub_scores, team_bb_scores, team_fx_scores, team_meet_scores, meet_hosts
    


In [137]:
# 5 Go to each of the meet's links and scrape the score information

# dfs we are adding to: meets_df, team_scores_df (meet_id, team_id), gymnast_scores_df (team_id, meet_id)

# Setting up the variables we will be using
meet_links = meets_df['link']
meet_ids = []
team_ids = []
team_meet_ids = []
team_vt_scores = []
team_ub_scores = []
team_bb_scores = []
team_fx_scores = []
team_meet_scores = []
gymnast_ids = []
gymnast_names = []
gymnast_team_ids = []
gymnast_meet_ids = []
gymnast_vt_scores = []
gymnast_ub_scores = []
gymnast_bb_scores = []
gymnast_fx_scores = []
gymnast_aa_scores = []
meet_hosts = []

meet_hosts

[]

In [121]:
meet_links[0:20]

0     https://roadtonationals.com/results/schedule/m...
1     https://roadtonationals.com/results/schedule/m...
2     https://roadtonationals.com/results/schedule/m...
3     https://roadtonationals.com/results/schedule/m...
4     https://roadtonationals.com/results/schedule/m...
5     https://roadtonationals.com/results/schedule/m...
6     https://roadtonationals.com/results/schedule/m...
7     https://roadtonationals.com/results/schedule/m...
8     https://roadtonationals.com/results/schedule/m...
9     https://roadtonationals.com/results/schedule/m...
10    https://roadtonationals.com/results/schedule/m...
11    https://roadtonationals.com/results/schedule/m...
12    https://roadtonationals.com/results/schedule/m...
13    https://roadtonationals.com/results/schedule/m...
14    https://roadtonationals.com/results/schedule/m...
15    https://roadtonationals.com/results/schedule/m...
16    https://roadtonationals.com/results/schedule/m...
17    https://roadtonationals.com/results/schedu

In [139]:
subset_meet_links = meet_links[0:100]

def get_all_the_team_results_from_all_the_meets(url):
    for meet_link in tqdm(subset_meet_links):
        #print(meet_link)
        get_team_score_info(meet_link)
    return team_ids, team_meet_ids, team_vt_scores, team_ub_scores, team_bb_scores, team_fx_scores, team_meet_scores, meet_hosts

get_all_the_team_results_from_all_the_meets(url)

print(len(team_ids))
print(len(team_meet_ids))
print(len(team_vt_scores))
print(len(team_ub_scores))
print(len(team_bb_scores))
print(len(team_fx_scores))
print(len(team_meet_scores))


team_results_df = pd.DataFrame({'team_id': team_ids, 'meet_id': team_meet_ids, 'vt_score': team_vt_scores, 'ub_score': team_ub_scores, 'bb_score': team_bb_scores, 'fx_score': team_fx_scores, 'meet_score': team_meet_scores})

  0%|          | 0/100 [00:00<?, ?it/s]

'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.A2FADF4611A9DC2965B21534D524FD54.e.118")>
'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.E9BFB17C850B6849CBDB48E3819F1C00.e.119")>
'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.65AF410A686771A97340593FAFD25501.e.120")>
'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A324CCA1A2641FC4D0D7ED1778.d.4E11673D8EFA2BC889CC4F1EDC97DAF1.e.121")>
'*****************************'
<selenium.webdriver.remote.webelement.WebElement (session="07009a743688c6649c21bd2153854f3e", element="f.ED1D65A

## Scraping using hidden APIS

In [ ]:
# # Replace with the actual API endpoint you discovered
# years = [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015]

# api_url = 'https://www.roadtonationals.com/api/women/finalresults/2024'

# # Include necessary headers, cookies, or auth tokens
# headers = {}

# response = requests.get(api_url, headers=headers)

# if response.status_code == 200:
#     data = response.json()
#     print(data)
# else:
#     print(f"Failed to retrieve data: {response.status_code}")